# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
from tld import get_tld

# Define directories

In [2]:
number = '18'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Open clean dataset

In [3]:
data_file = pd.read_csv(analyze_dir + 'csv_files/clean_files/clean_files_forum_' + number + '.csv')
data_file

,file_number,message_id
0,440001,message #985423
1,440002,message #985424
2,440003,message #985425
3,440005,message #985427
4,440006,message #985428
5,440007,message #985429
6,440009,message #985432
7,440010,message #985433
8,440011,message #985434
9,440012,message #985435


In [4]:
data_file.iloc[1][0]

440002

# Get links from files

In [5]:
print ("Forum files: %s" %(number))
url_data = []
for file_num in range(0,len(data_file)):
    sys.stdout.write('\rExtracting file number: %i' %(file_num+1) + ' / %i' %(len(data_file)))
    sys.stdout.flush()

    file = dataset + str(data_file.iloc[file_num][0])
    f = open(file)
    f = f.read()

    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [data_file.iloc[file_num][0], msid, title]
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a'):
                    username = usr.text
                    if username == '':
                        username = 'Eclipse User'

                    for link in userlink.find_all('span', attrs={'class':'MsgBodyText'}):
                        for atag in link.find_all('a'):
                            ss = atag.text.replace(' ','')
                            parser = []
                            try:
                                link = re.search('^https?:\/\/.*[\r\n]*[^.\\,:;]', ss)
                                link = link.group()
                                domain = urlparse(link)
                                tld_dt = get_tld(link)
                                parser = [username, link, domain.netloc, tld_dt]
                                data.insert(len(data),parser)
                            except:
                                pass

        n = 3
        link_name = []
        for i in range (0,len(data)-3):
            if data[n][1] not in link_name:
                url_data.append([data[0], data[1], data[2], data[n][0], data[n][1], data[n][2], data[n][3]])
                link_name.append(data[n][1])
            else:
                pass
            n = n + 1
    
    except:
        pass

print ('\nFiles extraction are finished')

Extracting file number: 212808 / 212808
Files extraction are finished


In [6]:
url_data

[[440001,
  'message #985423',
  'Welcome to eclipse.mihini',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [440001,
  'message #985423',
  'Welcome to eclipse.mihini',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.mihini',
  'www.eclipse.org',
  'org'],
 [440002,
  'message #985424',
  'Welcome to eclipse.damos',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [440002,
  'message #985424',
  'Welcome to eclipse.damos',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.damos',
  'www.eclipse.org',
  'org'],
 [440003,
  'message #985425',
  'Welcome to eclipse.incquery',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [440003,
  'message #985425',
  'Welcome to eclipse.incquery',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.incquery',
  'www.eclipse.org',
  'org'],
 [440005,
  'message #985427',
  '

In [7]:
with open (analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'topic', 'author', 'link', 'domain', 'top_level_domain']
    writers.writerow(header)
    for item in url_data:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
df_link = pd.read_csv(analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv')
df_link

,file_number,message_id,topic,author,link,domain,top_level_domain
0,440001,message #985423,Welcome to eclipse.mihini,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
1,440001,message #985423,Welcome to eclipse.mihini,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.mihini,www.eclipse.org,org
2,440002,message #985424,Welcome to eclipse.damos,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
3,440002,message #985424,Welcome to eclipse.damos,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.damos,www.eclipse.org,org
4,440003,message #985425,Welcome to eclipse.incquery,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
5,440003,message #985425,Welcome to eclipse.incquery,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.incquery,www.eclipse.org,org
6,440005,message #985427,Welcome to eclipse.mihini,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
7,440005,message #985427,Welcome to eclipse.mihini,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.mihini,www.eclipse.org,org
8,440006,message #985428,Welcome to eclipse.damos,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
9,440006,message #985428,Welcome to eclipse.damos,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.damos,www.eclipse.org,org


# Remove links if the author is Eclipse Webmaster

In [9]:
cols = ['file_number','message_id','topic','author','link','domain','top_level_domain']
new_df_link = df_link[cols][df_link[cols]['author'] != 'Eclipse Webmaster']
new_df_link

,file_number,message_id,topic,author,link,domain,top_level_domain
18,440012,message #985435,Eclipse complains about working code,Axel Mueller,http://wiki.eclipse.org/CDT/User/FAQ#CDT_does_...,wiki.eclipse.org,org
31,440021,message #985445,Change web service response,Russell Bateman,http://www.coderanch.com/forums/f-51/Web-Services,www.coderanch.com,com
50,440035,message #985460,Resolving Crossreferences in Eclipse context,Christian Dietrich,http://xtext.itemis.com,xtext.itemis.com,com
87,440060,message #985488,Emma Update,Flavio Donze,http://download.eclipse.org/tools/buckminster/...,download.eclipse.org,org
88,440060,message #985488,Emma Update,Flavio Donze,http://www.eclemma.org/changes.html,www.eclemma.org,org
89,440060,message #985488,Emma Update,Bernard Sarter,https://bugs.eclipse.org/bugs/show_bug.cgi?id=...,bugs.eclipse.org,org
90,440060,message #985488,Emma Update,Thomas Hallgren,http://download.cloudsmith.com/buckminster/ext...,download.cloudsmith.com,com
91,440060,message #985488,Emma Update,Flavio Donze,https://issues.jenkins-ci.org/browse/JENKINS-1...,issues.jenkins-ci.org,org
92,440060,message #985488,Emma Update,Lorenzo Bettini,http://www.lorenzobettini.it,www.lorenzobettini.it,it
93,440060,message #985488,Emma Update,Lorenzo Bettini,https://groups.google.com/forum/#!topic/jenkin...,groups.google.com,com


In [10]:
new_df_link.to_csv(analyze_dir + 'csv_files/links_list/cleaned/links_without_bot_authors_forum_' + number + '.csv', index=False)
print ("CSV file has been created")

CSV file has been created
